In [ ]:
#connect with gg drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import tensorflow as tf
# print(tf.__version__)

2.17.0


In [ ]:
import keras
print(keras.__version__)

3.4.1


In [ ]:
# Import thư viện
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation, GRU, Dense, Input, Add, Concatenate, Reshape,Lambda, BatchNormalization, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import SGD, RMSprop
import pickle
import keras

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/tokenizer1.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/sequences_digit1.pkl', 'rb') as f:
    sequences_digit = pickle.load(f)

In [ ]:
print(len(tokenizer.word_index))

32988


In [ ]:

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

32989


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/weights-training-languagemodelnew.keras'
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
import numpy as np
import tensorflow
vocab_size = len(tokenizer.word_index) + 1
def data_generator(sequences_digit, batch_size):
    while True:
        batch_paths = np.random.choice(a = len(sequences_digit), size = batch_size)
        input = []
        output = []
        for i in batch_paths:
            input.append(sequences_digit[i][:-1])
            output.append(sequences_digit[i][-1])
        input = np.array(input)
        output = tensorflow.keras.utils.to_categorical(output, num_classes=vocab_size) # Now tensorflow is defined
        output = np.array(output)
        yield (input, output)

In [ ]:
# Khởi tạo mô hình trước khi tải trọng số
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=50))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(vocab_size, activation='softmax'))
model.build(input_shape=(None, 50))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 50, 50)              │       1,649,450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 50, 50)              │             200 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 50, 512)             │       1,153,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 512)                 │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          51,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32989)               │       3,331,889 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,285,463 (31.61 MB)

 Trainable params: 8,285,163 (31.61 MB)

 Non-trainable params: 300 (1.17 KB)

In [ ]:
print("Loading pre-trained weight")
model.load_weights('/content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/weights-training-languagemodelnew.keras')

Loading pre-trained weight


In [ ]:
batch_size=512

In [ ]:
model.fit(x=data_generator(sequences_digit, batch_size), steps_per_epoch=(len(sequences_digit)//batch_size) ,
          epochs=100, verbose=1, callbacks=callbacks_list)

Epoch 1/100
1953/1953 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.4579 - loss: 2.3680
Epoch 1: accuracy improved from -inf to 0.45806, saving model to /content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/weights-training-languagemodelnew.keras
1953/1953 ━━━━━━━━━━━━━━━━━━━━ 512s 261ms/step - accuracy: 0.4579 - loss: 2.3680
Epoch 2/100
1953/1953 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.4612 - loss: 2.3664
Epoch 2: accuracy improved from 0.45806 to 0.46315, saving model to /content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/weights-training-languagemodelnew.keras
1953/1953 ━━━━━━━━━━━━━━━━━━━━ 508s 260ms/step - accuracy: 0.4612 - loss: 2.3664
Epoch 3/100
 696/1953 ━━━━━━━━━━━━━━━━━━━━ 5:27 261ms/step - accuracy: 0.4668 - loss: 2.3354

In [ ]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/doAn_ChuyenNganh/weights-training-languagemodel.h5')

In [ ]:
!pip install pyvi # Install the missing 'pyvi' module.
from pyvi import ViTokenizer
from keras.preprocessing.sequence import pad_sequences
import string

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.4 MB/s eta 0:00:00


Không cần điịnh nghĩa mạng khi đã lưu .h5

In [ ]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    doc = doc.lower() #Lower
    tokens = doc.split() #Split in_to words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

In [ ]:

import numpy as np

def preprocess_input(doc):
    tokens = clean_document(doc)
     # Đưa tokens vào dưới dạng danh sách
    tokens = tokenizer.texts_to_sequences([tokens])
     # padding để phù hợp với độ dài 50
    tokens = pad_sequences(tokens, maxlen=50, truncating='pre')
    # Đảm bảo tokens là mảng có shape (1, 50)
    return np.reshape(tokens, (1, 50))

In [ ]:
def generate_text(text_input, n_words):
    tokens = preprocess_input(text_input)
    for _ in range(n_words):
       # Dự đoán xác suất từ tiếp theo
        predictions = model.predict(tokens)
        # Lấy từ có xác suất cao nhất
        next_digit = np.argmax(predictions, axis=-1)
        # Thêm từ dự đoán vào chuỗi
        tokens = np.append(tokens, next_digit)
         # Chỉ giữ lại 50 từ cuối cùng
        tokens = tokens[-50:]
        # Định hình lại chuỗi thành (1, 50)
        tokens = np.reshape(tokens, (1, 50))
    # Mapping từ chỉ số số học sang từ
    tokens = np.reshape(tokens, (50))
    out_word = []
    for token in tokens:
        for word, index in tokenizer.word_index.items():
            if index == token:
                out_word.append(word)
                break
    return ' '.join(out_word)  # Trả về chuỗi văn bản kết quả

In [ ]:
# Test
seq = 'hôm nay tôi đi'
generate_text(seq, 25)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'hôm_nay tôi đi xem trong 5 tháng thú_vui thú_vị là 150 người ngoại sống ở phía nam nhỏ và 6 tập tranh biểu_diễn làm_việc tại chợ rừng ở'

In [ ]:
# Test
seq1 = 'mùa thu hà nội rất'
generate_text(seq1, 25)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


'mùa thu hà_nội rất nhiều gấp đôi số tháng lãnh_đạo 300 vua được thể_hiện ở tokyo có số điểm dài quan_trọng hơn một năm là một triệu bản và'

In [ ]:
# Test
seq2 = 'đường phố ở việt nam'
generate_text(seq2, 25)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


'đường_phố ở việt_nam theo hình hẹp sàn chân ba lệch hướng đến một bờ sông và thuyền cắm cắt_lớp tiếng gọi của xí_nghiệp thành_phố saal quỷ hiện thành'